# 3. Training and Testing Our CNN

In the first notebook, we downloaded and prepared the Fashion-MNIST data to be used as input for our CNN.

In the second notebook, we designed the architecture of the CNN, defining the feedforward layers, the loss function and the optimisation (backpropagation) method.

In this notebook, we will train our data on the `train_set` (with the help of our `validation_set`) and also test the accuracy of our model on the `test_set`.

### Outline of this notebook

>1. Preliminaries for training
    - 1.1: `train` function
    - 1.2: `test` function
    - 1.3: Check accuracy of model before training
<br>
<br>
2. Train the model
<br>
<br>
3. Test the model

We first load the data and define the CNN. Refer to the first two notebooks for detailed walkthroughs of the following code.

In [1]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

#### Load the data

train_set = datasets.FashionMNIST(root = 'FashionMNIST_data', train = True, download = True)
test_set = datasets.FashionMNIST(root = 'FashionMNIST_data', train = False, download = True)

validation = 0.2

training_size = len(train_set)
indices = list(range(training_size))
np.random.shuffle(indices)
split = int(np.floor(validation * training_size))
train_index = indices[:split]
validation_index = indices[split:]
train_sampler = SubsetRandomSampler(train_index)
validation_sampler = SubsetRandomSampler(validation_index)

train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, ), (0.5, ))])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, ), (0.5, ))])

train_set = datasets.FashionMNIST(root='./FashionMNIST_data', train=True, download=False, transform=train_transform)
test_set = datasets.FashionMNIST(root='./FashionMNIST_data', train=False, download=False, transform=test_transform)

batch_size = 64

train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size = batch_size,
                                           sampler = train_sampler, 
                                           shuffle = False)

validation_loader = torch.utils.data.DataLoader(train_set, 
                                                batch_size = batch_size, 
                                                sampler = validation_sampler, 
                                                shuffle = False)

test_loader = torch.utils.data.DataLoader(test_set, 
                                          batch_size = batch_size, 
                                          shuffle = True)

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

#### Design the CNN architecture

num_filters = 10
kernel_size = 3
padding = 1

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        # Convolutional layer 1 (sees 28x28x1 image tensor)
        self.conv1 = nn.Conv2d(1, num_filters, kernel_size, padding = padding)
        # Convolutional layer 2 (sees a 14x14x10 tensor)
        self.conv2 = nn.Conv2d(14, 20, kernel_size, padding = padding)
        # Maxpooling layer of size 2x2
        self.pool = nn.MaxPool2d(2, 2)
        # Fully-connected linear layer 1 (sees a 7x7x20 tensor -> 300)
        self.fc1 = nn.Linear(7 * 7 * 20, 300)
        # Fully-connected linear layer 2 (300 -> 10)
        self.fc2 = nn.Linear(300, 10)
        # Dropout layer (p=0.2)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        # Design sequence of convolutional and pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # Flatten image input into a row vector to feed into the fully-connected layers
        x = x.view(-1, 7 * 7 * 20)
        # Add fully-connected layer 1 with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        # Add fully-connected layer 2 with dropout
        x = self.dropout(F.relu(self.fc2(x)))
        return x
    
model = CNN()

criterion = nn.CrossEntropyLoss()

optimiser = torch.optim.SGD(model.parameters(), lr=0.03)

---

### 1. Prelimiaries for training

#### 1.1: Define the `train` function

We start by defining a `train` function that will be called when we train our CNN. We will train over a number of `epochs`, which is the number of times that we will cycle through the entire `train_set`.

The sequence of steps that is performed for each epoch is:
>1. Train with the `train_set`:
    - Step 1: Zero's the gradients to prepare for feedforward
    - Step 2: Feedforward the input image through the network
    - Step 3: Compute the loss relative to `label`
    - Step 4: Propagate gradients back into the network's parameters (i.e. calculate the gradient with of the loss function with respect to each parameter)
    - Step 5: Optimise / update the parameters
    - Step 6: Sum the `training_loss`
<br>
<br>
2. Evaluate with the `validation_set`:
    - Step 1: Feedforward the image through the trained model
    - Step 2: Compute the loss
    - Step 3: Sum the `validation_loss`
<br>
<br>
3. Then, after each epoch, we print the average `training_loss` and `validation_loss` for that epoch.

Additionally, from one epoch to another, we keep track of the average validation loss. We save the model as `CNN_FashionMNIST.pt` each time our model attains a new minimum average validation loss for an epoch.

Also note that we check if CUDA is available on the machine that we are using. If it is available, we are able to train our CNN on the machine's GPU, else the training is done on the CPU.

In [2]:
# Define train function

def train(n_epochs):
    
    # Set minimum validation loss as infinity.
    # This is used to check and save the model.
    validation_loss_min = np.Inf
    
    # Iterate over train_set for n_epochs times
    for epoch in range(n_epochs):
        
        # Keep track of training and validation loss
        total_training_loss = 0
        total_validation_loss = 0
        
        # ----- Train the model ------
        model.train()
        
        for image, label in train_loader:
            # Move tensors to GPU if CUDA is available
            if train_on_gpu:
                image, label = image.cuda(), label.cuda()
            # Step 1: Zero the gradients
            optimiser.zero_grad()
            # Step 2: Feedforward the input image
            output = model(image)
            # Step 3: Compute the loss
            loss = criterion(output, label)
            # Step 4: Backpropagation
            loss.backward()
            # Step 5: Update parameters
            optimiser.step()
            # Step 6: Update training loss
            total_training_loss += loss.item()

        # ----- Validate the model ------
        
        # Turn off gradients for validation
        model.eval()
        
        for image, label in validation_loader:
            # Move tensors to GPU if CUDA is available
            if train_on_gpu:
                image, label = image.cuda(), label.cuda()
            # Step 1: Feedforward: Compute predicted outputs after
            # training on the train_set above
            output = model(image)
            # Step 2: Compute loss
            loss = criterion(output, label)
            # Step 3: Update validation loss
            total_validation_loss += loss.item()
            
        # Compute average losses in this epoch
        avg_training_loss = total_training_loss / len(train_loader.sampler)
        avg_validation_loss = total_validation_loss / len(validation_loader.sampler)
        
        # Print average training and validation loss in this epoch
        print("Epoch: {} out of {}".format(epoch, n_epochs))
        print("Average training loss: {:.6f}".format(avg_training_loss))
        print("Average validation loss: {:.6f}".format(avg_validation_loss))
        
        # ------ Save model if validation loss has decreased ------
        if avg_validation_loss <= validation_loss_min:
            print("Validation loss decreased from {} in the previous epoch to {} in this epoch.".format(validation_loss_min, avg_validation_loss))
            print()
            torch.save(model.state_dict(), 'CNN_FashionMNIST.pt')
            validation_loss_min = avg_validation_loss

#### 1.2 Define the `test` function

Below, we define the `test` function that we will call when testing our model on unseen images in the `test_set`.


In [4]:
# Define test function

def test(test_loader):
    
    # Contruct dictionaries to count the number of correct and
    # incorrect predictions for each class
    correct_dict = dict(zip(classes, [0]*len(classes)))
    incorrect_dict = dict(zip(classes, [0]*len(classes)))
    
    # Track test loss
    total_test_loss = 0
    
    # Load the saved model with the lowest validation loss
    model.load_state_dict(torch.load('CNN_FashionMNIST.pt'))
    
    # ------ Test the model ------
    
    # Turn off gradients for validation
    model.eval()
    
    for image, label in test_loader:
        # Move tensors to GPU if CUDA is available
        if train_on_gpu:
            image, label = image.cuda(), label.cuda()
        # Step 1: Feedforward
        output = model(image)
        # Step 2: Compute loss
        loss = criterion(output, label)
        # Step 3: Update test loss
        total_test_loss += loss.item()
        # Step 4: Convert output values to log probabilities using logSoftmax
        log_probs = F.log_softmax(output, dim = 1)
        # Step 5: Convert log probabilities to probabilities
        probs = torch.exp(log_probs)
        # Step 6: Determine the predicted class
        top_prob, top_prob_class = probs.topk(1, dim=1)
        if top_prob_class == label[0]:
            
    
    
    # Compute average test loss
    avg_test_loss = total_test_loss / len(test_loader.dataset)
    print("Test loss: {:.6f}".format(avg_test_loss))

#### 1.3: Check accuracy of model before training

Before we proceed to train our model, it is interesting to check out the accuracy of our CNN. Since the parameters of our CNN are randomly generated, we should expect the accuracy to be around 1/10 (i.e. 10%), implying that the prediction of our untrained model is as good as a random guess!
